In [2]:
import pandas as pd
from google import genai
from dotenv import load_dotenv
import os
from openai import OpenAI
import json
import time
import re


In [3]:


load_dotenv()
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")




## Load the data set

In [4]:
path = os.path.join("data", "All_capped_keywords.csv")
df = pd.read_csv(path)

df.head()

,paperID,venue,year,openAccessPdf,url,authors,referenceCount,title,abstract,conclusion,...,Effectiveness,Future Works,Sentiment,Sentiment Score,combined,combined_keywords,response_keywords,future_work_keywords,capped_keywords,field
0,000194903cb83bd4af714f950d0266382e2772fc,AAAI Conference on Artificial Intelligence,2021,True,https://ojs.aaai.org/index.php/AAAI/article/do...,"Zhihong Chen (2144309151),Taiping Yao (5130095...",35,Generalizable Representation Learning for Mixt...,Face anti-spoofing approach based on domain ge...,"conclusion:As shown in Figure 3, Tables 1 and2...",...,['Highly effective'],['Further enhance interpretability and visuali...,['Positive'],['8.5'],Generalizable Representation Learning for Mixt...,"[('spoofing meta learning', 0.6263), ('face an...","[('meta learning domain', 0.5346), ('face anti...","[('face anti spoofing', 0.6095), ('anti spoofi...","[('spoofing meta learning', 0.6263), ('face an...",Artificial Intelligence
1,000351d89ee73f2fa721961129ec9c0758cf20ca,AAAI Conference on Artificial Intelligence,2017,False,NaN,"Elias Fernández Domingos (9563986),J. C. Burgu...",9,Coordinating Human and Agent Behavior in Colle...,\n \n Various social situations entail a colle...,NaN,...,['Moderately effective'],['Explore further applications of anticipatory...,['The use of anticipative agents and recurrent...,['8'],Coordinating Human and Agent Behavior in Colle...,"[('collective risk game', 0.5749), ('agent beh...","[('collective risk game', 0.5331), ('agents co...","[('applications anticipatory behavior', 0.6232...","[('collective risk game', 0.5749), ('agent beh...",Artificial Intelligence
2,001198d3ef0d304718cfd64ff241d0133adcf928,AAAI Conference on Artificial Intelligence,2017,True,https://ojs.aaai.org/index.php/AAAI/article/do...,"Cao Xiao (145781464),Ping Zhang (2154286366),W...",29,Adverse Drug Reaction Prediction with Symbolic...,\n \n Adverse drug reaction (ADR) is a major b...,"conclusion:In this paper, we presented three L...",...,['Highly effective in improving prediction acc...,['Improvement of the medical ontology system w...,['The proposed models show promise in enhancin...,['8.5'],Adverse Drug Reaction Prediction with Symbolic...,"[('drug reaction prediction', 0.5413), ('adr p...","[('drug adr data', 0.5261), ('lda adr predicti...","[('improvement medical ontology', 0.7167), ('m...","[('drug reaction prediction', 0.5413), ('adr p...",Artificial Intelligence
3,00133d41d5ecef1a9d046d2b92bb2a23a335cb7c,AAAI Conference on Artificial Intelligence,2020,True,https://ojs.aaai.org/index.php/AAAI/article/do...,"Yanchao Sun (120738683),Xiangyu Yin (31186224)...",37,TempLe: Learning Template of Transitions for S...,Transferring knowledge among various environme...,"conclusion:In this work, we propose TempLe, th...",...,['Highly effective'],['Investigating transition probability and rew...,['Positive'],['8.5'],TempLe: Learning Template of Transitions for S...,"[('learning multiple tasks', 0.5994), ('multi ...","[('learns multiple tasks', 0.6435), ('learning...","[('similarities continuous mdp', 0.551), ('mod...","[('learning multiple tasks', 0.5994), ('multi ...",Artificial Intelligence
4,002285ade41b9e1313f9c914b99e68b62fab7ff1,AAAI Conference on Artificial Intelligence,2017,True,https://aaai.org/ojs/index.php/AAAI/article/do...,"M. Riemer (40497459),Tim Klinger (2652016),Dja...",62,Scalable Recollections for Continual Lifelong ...,Given the recent success of Deep Learning appl...,NaN,...,['Moderately effective'],['Exploring further improvements in training e...,['Positive'],['8'],Scalable Recollections for Continual Lifelong ...,"[('continual lifelong learning', 0.7155), ('re...","[('continual lifelong learning', 0.6788), ('li...","[('exploring improvements training', 0.6438), ...","[('continual lifelong learning', 0.7155), ('re...",Artificial Intelligence


### columns

In [4]:
df.columns

Index(['paperID', 'venue', 'year', 'openAccessPdf', 'url', 'authors',
       'referenceCount', 'title', 'abstract', 'conclusion', 'Chatgpt Response',
       'Key Takeaways', 'Importance', 'Model/Method Proposed', 'Performance',
       'Effectiveness', 'Future Works', 'Sentiment', 'Sentiment Score',
       'combined', 'combined_keywords', 'response_keywords',
       'future_work_keywords', 'capped_keywords', 'field'],
      dtype='object')

## Explore a data

In [5]:
df.shape

(91919, 25)

In [7]:
first_row = df.iloc[7]

first_row

paperID                           002b12ad36d3b19a78d0754e6949f791987830dd
venue                           AAAI Conference on Artificial Intelligence
year                                                                  2020
openAccessPdf                                                         True
url                      https://ojs.aaai.org/index.php/AAAI/article/do...
authors                  Qingdong He (153075429),Zhengning Wang (152764...
referenceCount                                                          48
title                    SVGA-Net: Sparse Voxel-Graph Attention Network...
abstract                 Accurate 3D object detection from point clouds...
conclusion               conclusion:In this paper, we propose a novel s...
Chatgpt Response         Key Takeaways:\n- Sparse Voxel-Graph Attention...
Key Takeaways            ['Sparse Voxel-Graph Attention Network (SVGA-N...
Importance               ['Crucial for accurate 3D object detection in ...
Model/Method Proposed    

In [8]:
print(first_row['title'])
print("\n\n")
print(first_row['abstract'])

print("\n\n")


print(first_row['combined'])

SVGA-Net: Sparse Voxel-Graph Attention Network for 3D Object Detection from Point Clouds



Accurate 3D object detection from point clouds has become a crucial component in autonomous driving. However, the volumetric representations and the projection methods in previous works fail to establish the relationships between the local point sets. In this paper, we propose Sparse Voxel-Graph Attention Network (SVGA-Net), a novel end-to-end trainable network which mainly contains voxel-graph module and sparse-to-dense regression module to achieve comparable 3D detection tasks from raw LIDAR data. Specifically, SVGA-Net constructs the local complete graph within each divided 3D spherical voxel and global KNN graph through all voxels. The local and global graphs serve as the attention mechanism to enhance the extracted features. In addition, the novel sparse-to-dense regression module enhances the 3D box estimation accuracy through feature maps aggregation at different levels. Experiments on KI

## keywords


In [22]:
df['combined_keywords'].head(5)

0    [('spoofing meta learning', 0.6263), ('face an...
1    [('collective risk game', 0.5749), ('agent beh...
2    [('drug reaction prediction', 0.5413), ('adr p...
3    [('learning multiple tasks', 0.5994), ('multi ...
4    [('continual lifelong learning', 0.7155), ('re...
Name: combined_keywords, dtype: object

## dispay the title, abstract, and keywords

In [8]:
df[['title', 'abstract', 'combined_keywords']].head()



,title,abstract,combined_keywords
0,Generalizable Representation Learning for Mixt...,Face anti-spoofing approach based on domain ge...,"[('spoofing meta learning', 0.6263), ('face an..."
1,Coordinating Human and Agent Behavior in Colle...,\n \n Various social situations entail a colle...,"[('collective risk game', 0.5749), ('agent beh..."
2,Adverse Drug Reaction Prediction with Symbolic...,\n \n Adverse drug reaction (ADR) is a major b...,"[('drug reaction prediction', 0.5413), ('adr p..."
3,TempLe: Learning Template of Transitions for S...,Transferring knowledge among various environme...,"[('learning multiple tasks', 0.5994), ('multi ..."
4,Scalable Recollections for Continual Lifelong ...,Given the recent success of Deep Learning appl...,"[('continual lifelong learning', 0.7155), ('re..."


In [ ]:
keyword_df = df.copy()
keyword_df['title_sentiment_keyword'] = 'Unknown'

keyword_df["title_sentiment_keyword"]

0        Unknown
1        Unknown
2        Unknown
3        Unknown
4        Unknown
          ...   
91914    Unknown
91915    Unknown
91916    Unknown
91917    Unknown
91918    Unknown
Name: title_sentiment_keyword, Length: 91919, dtype: object

# List of filter tags

In [21]:
filters = []
with open("regex_filters.json", "r") as f:
    filters = json.load(f)

filters

{'funny': ['fun(ny|nier|niest)',
  'laugh(able|ing)?',
  '(comic|comical)',
  'jest(ing)?',
  '(pun|puns|punning)',
  'tongue[- ]?in[- ]?cheek'],
 'playful': ['playful(ly|ness)?',
  'whimsic(al|ally)?',
  'light[- ]?hearted',
  'mischiev(ous|ously)',
  'joll(y|iness)',
  'frolic(some)?',
  'quirk(y|iness)'],
 'creative': ['creativ(e|ity|ely)',
  'imagin(ative|ation)',
  'innovat(ive|ion)',
  'invent(ive|ion)',
  'novel(ty)?',
  'original(ly)?',
  'clever(ness)?',
  'ingenious',
  'visionar(y|ies)'],
 'catchy': ['catch(y|ier|iest)',
  'memorabl(e|y)',
  'snapp(y|ier|iest)',
  'quirk(y|ier|iest)',
  'striking',
  'attention[- ]?grab(bing|ber)?',
  'punch(y|ier|iest)',
  'eye[- ]?catch(ing|er)?'],
 'humor': ['humou?r(ous|ist|ists)?',
  'witt(y|iness)?',
  'sarcasm(ic|ically)?',
  'satir(e|ical|ist|ists)?',
  'iron(y|ic|ically)?',
  'quip(s)?',
  'banter(ing)?',
  'parody',
  'spoof',
  'lampoon'],
 'hilarious': ['hilarious(ly)?',
  'uproar(ious|iously)',
  'side[- ]?splitting',
  'knee[- 

## use keywords to add filters to the titles

In [9]:
def keyword_title_categorize(regex_filters, title):
    title = title.lower()
    for filter, regex_patterns in regex_filters.items():
        for pattern in regex_patterns:
            if re.search(pattern, title):
                return filter
    return "academic" # default category is academic

## update the **title_sentiment_keyword** column based on the filter

In [ ]:
for index, row in keyword_df.iterrows():
    filter = keyword_title_categorize(filters, row['title'])

    keyword_df.at[index, 'title_sentiment_keyword'] = filter
 
    
keyword_df['title_sentiment_keyword'].value_counts()





Index(['paperID', 'venue', 'year', 'openAccessPdf', 'url', 'authors',
       'referenceCount', 'title', 'abstract', 'conclusion', 'Chatgpt Response',
       'Key Takeaways', 'Importance', 'Model/Method Proposed', 'Performance',
       'Effectiveness', 'Future Works', 'Sentiment', 'Sentiment Score',
       'combined', 'combined_keywords', 'response_keywords',
       'future_work_keywords', 'capped_keywords', 'field',
       'title_sentiment_keyword'],
      dtype='object')

## save the data frame

In [42]:

keyword_df.to_csv("data/keywords_categorized_data.csv", index=False)

## copy the data frame 

In [10]:
tagged_df = df

tagged_df['model_title_sentiment'] = "Unknown"


tagged_df.head()


,paperID,venue,year,openAccessPdf,url,authors,referenceCount,title,abstract,conclusion,...,Future Works,Sentiment,Sentiment Score,combined,combined_keywords,response_keywords,future_work_keywords,capped_keywords,field,model_title_sentiment
0,000194903cb83bd4af714f950d0266382e2772fc,AAAI Conference on Artificial Intelligence,2021,True,https://ojs.aaai.org/index.php/AAAI/article/do...,"Zhihong Chen (2144309151),Taiping Yao (5130095...",35,Generalizable Representation Learning for Mixt...,Face anti-spoofing approach based on domain ge...,"conclusion:As shown in Figure 3, Tables 1 and2...",...,['Further enhance interpretability and visuali...,['Positive'],['8.5'],Generalizable Representation Learning for Mixt...,"[('spoofing meta learning', 0.6263), ('face an...","[('meta learning domain', 0.5346), ('face anti...","[('face anti spoofing', 0.6095), ('anti spoofi...","[('spoofing meta learning', 0.6263), ('face an...",Artificial Intelligence,Unknown
1,000351d89ee73f2fa721961129ec9c0758cf20ca,AAAI Conference on Artificial Intelligence,2017,False,NaN,"Elias Fernández Domingos (9563986),J. C. Burgu...",9,Coordinating Human and Agent Behavior in Colle...,\n \n Various social situations entail a colle...,NaN,...,['Explore further applications of anticipatory...,['The use of anticipative agents and recurrent...,['8'],Coordinating Human and Agent Behavior in Colle...,"[('collective risk game', 0.5749), ('agent beh...","[('collective risk game', 0.5331), ('agents co...","[('applications anticipatory behavior', 0.6232...","[('collective risk game', 0.5749), ('agent beh...",Artificial Intelligence,Unknown
2,001198d3ef0d304718cfd64ff241d0133adcf928,AAAI Conference on Artificial Intelligence,2017,True,https://ojs.aaai.org/index.php/AAAI/article/do...,"Cao Xiao (145781464),Ping Zhang (2154286366),W...",29,Adverse Drug Reaction Prediction with Symbolic...,\n \n Adverse drug reaction (ADR) is a major b...,"conclusion:In this paper, we presented three L...",...,['Improvement of the medical ontology system w...,['The proposed models show promise in enhancin...,['8.5'],Adverse Drug Reaction Prediction with Symbolic...,"[('drug reaction prediction', 0.5413), ('adr p...","[('drug adr data', 0.5261), ('lda adr predicti...","[('improvement medical ontology', 0.7167), ('m...","[('drug reaction prediction', 0.5413), ('adr p...",Artificial Intelligence,Unknown
3,00133d41d5ecef1a9d046d2b92bb2a23a335cb7c,AAAI Conference on Artificial Intelligence,2020,True,https://ojs.aaai.org/index.php/AAAI/article/do...,"Yanchao Sun (120738683),Xiangyu Yin (31186224)...",37,TempLe: Learning Template of Transitions for S...,Transferring knowledge among various environme...,"conclusion:In this work, we propose TempLe, th...",...,['Investigating transition probability and rew...,['Positive'],['8.5'],TempLe: Learning Template of Transitions for S...,"[('learning multiple tasks', 0.5994), ('multi ...","[('learns multiple tasks', 0.6435), ('learning...","[('similarities continuous mdp', 0.551), ('mod...","[('learning multiple tasks', 0.5994), ('multi ...",Artificial Intelligence,Unknown
4,002285ade41b9e1313f9c914b99e68b62fab7ff1,AAAI Conference on Artificial Intelligence,2017,True,https://aaai.org/ojs/index.php/AAAI/article/do...,"M. Riemer (40497459),Tim Klinger (2652016),Dja...",62,Scalable Recollections for Continual Lifelong ...,Given the recent success of Deep Learning appl...,NaN,...,['Exploring further improvements in training e...,['Positive'],['8'],Scalable Recollections for Continual Lifelong ...,"[('continual lifelong learning', 0.7155), ('re...","[('continual lifelong learning', 0.6788), ('li...","[('exploring improvements training', 0.6438), ...","[('continual lifelong learning', 0.7155), ('re...",Artificial Intelligence,Unknown


## classify a single research title using Gemini model for NLP

In [ ]:
def apply_single_tone_filter(paper_title):
    
    client = genai.Client(api_key=GEMINI_API_KEY)
    try:

        prompt = (
            "You are LLM model responsible for applying filters to research paper titles based on their sentiment.\n\n"
            f"Given the following research paper title: {paper_title}\n"
            f"from this list of filters: {filters}, what is the most appropriate tone?"
            f"Respond ONLY with a single string, e.g.: \"funny\"\n"
            f"Do not include explanations or extra text."
        )
        response = client.models.generate_content(
            model="gemini-2.5-flash",
            contents=prompt,
        )
        response_text = response.text.strip()
        return response_text


    except Exception as e:
        print(f"Error occurred: {e}")
    finally:
        time.sleep(1)




In [15]:
filters = [
    'funny',
    'creative',
    'playful',
    'academic',
    'informal',
    'amusing',
    'absurd',
    'silly',
    'catcy',
    'amusing'
]


## Function uses gemini to apply filters to  research paper titles

In [18]:
def apply_tone_filter(paper_title):
    
    client = genai.Client(api_key=GEMINI_API_KEY)
    try:

        prompt = (
            "You are LLM model responsible for applying filters to research paper titles based on their sentiment.\n\n"
            f"Given the following research paper titles: {paper_title}\n"
            f"from this list of filters: {filters}, what is the most appropriate tone?"
            f"Respond ONLY with a single word e.g 'funny'\n"
            f"Do not include explanations or extra text."
        )
        response = client.models.generate_content(
            model="gemini-2.5-flash",
            contents=prompt,
        )
        return response.text.strip()
        
    except Exception as e:
        
        print(f"Error occurred: {e}")
        return 'unknown'
    f


print(apply_tone_filter('dancing is funny'))



funny


# add filters to the data frame that classify the research paper title

In [ ]:


def apply_sentiment_filters(start, end, data):
    for i in range(start, end):
        title = data.at[i, 'title']  # get the title at row i
        sentiment = apply_tone_filter(title)  
        data.at[i, 'model_title_sentiment'] = sentiment  

       


# try with the first 50 rows
apply_sentiment_filters(0, 50, tagged_df)


In [35]:
df['title'].tolist()

['Generalizable Representation Learning for Mixture Domain Face Anti-Spoofing',
 'Coordinating Human and Agent Behavior in Collective-Risk Scenarios',
 'Adverse Drug Reaction Prediction with Symbolic Latent Dirichlet Allocation',
 'TempLe: Learning Template of Transitions for Sample Efficient Multi-task RL',
 'Scalable Recollections for Continual Lifelong Learning',
 'Cubic LSTMs for Video Prediction',
 'MarioNETte: Few-shot Face Reenactment Preserving Identity of Unseen Targets',
 'SVGA-Net: Sparse Voxel-Graph Attention Network for 3D Object Detection from Point Clouds',
 'On The Verification of Neural ODEs with Stochastic Guarantees',
 'Discovering New Intents with Deep Aligned Clustering',
 'Social Choice Under Metric Preferences: Scoring Rules and STV',
 'Incentive-Aware PAC Learning',
 'A New Framework for Online Testing of Heterogeneous Treatment Effect',
 'Combining Fact Extraction and Verification with Neural Semantic Matching Networks',
 'Optimal Auction Based Automated Negoti